In [1]:
import os
import io
import sys
import random
import math
import numpy as np
import skimage.io
import matplotlib
import matplotlib.pyplot as plt
import logging
import aiohttp
import asyncio
from aiohttp import web, hdrs
from typing import ByteString
from PIL import Image
from concurrent.futures import ThreadPoolExecutor

# Set log detail level
LOGGING_LEVEL = logging.INFO

# Root directory of the project
ROOT_DIR = os.path.abspath("../")

# Import Mask RCNN
sys.path.append(ROOT_DIR)  # To find local version of the library
from mrcnn import utils
import mrcnn.model as modellib
from mrcnn import visualize
# Import COCO config
sys.path.append(os.path.join(ROOT_DIR, "samples/coco/"))  # To find local version
from samples.coco import coco

%matplotlib inline 

# Directory to save logs and trained model
MODEL_DIR = os.path.join(ROOT_DIR, "logs")

# Local path to trained weights file
COCO_MODEL_PATH = os.path.join(ROOT_DIR, "mask_rcnn_coco.h5")
# Download COCO trained weights from Releases if needed
if not os.path.exists(COCO_MODEL_PATH):
    utils.download_trained_weights(COCO_MODEL_PATH)

# Directory of images to run detection on
IMAGE_DIR = os.path.join(ROOT_DIR, "Mask_RCNN/images")


class InferenceConfig(coco.CocoConfig):
    # Set batch size to 1 since we'll be running inference on
    # one image at a time. Batch size = GPU_COUNT * IMAGES_PER_GPU
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1

config = InferenceConfig()
config.display()

# Create model object in inference mode.
model = modellib.MaskRCNN(mode="inference", model_dir=MODEL_DIR, config=config)

# Load weights trained on MS-COCO
model.load_weights(COCO_MODEL_PATH, by_name=True)


# COCO Class names
# Index of the class in the list is its ID. For example, to get ID of
# the teddy bear class, use: class_names.index('teddy bear')
class_names = ['BG', 'Nonce', 'bicycle', 'car', 'motorcycle', 'airplane',
               'bus', 'train', 'truck', 'boat', 'traffic light',
               'fire hydrant', 'stop sign', 'parking meter', 'bench', 'bird',
               'cat', 'dog', 'horse', 'sheep', 'cow', 'elephant', 'bear',
               'zebra', 'giraffe', 'backpack', 'umbrella', 'handbag', 'tie',
               'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball',
               'kite', 'baseball bat', 'baseball glove', 'skateboard',
               'surfboard', 'tennis racket', 'bottle', 'wine glass', 'cup',
               'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple',
               'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza',
               'donut', 'cake', 'chair', 'couch', 'potted plant', 'bed',
               'dining table', 'toilet', 'tv', 'laptop', 'mouse', 'remote',
               'keyboard', 'cell phone', 'microwave', 'oven', 'toaster',
               'sink', 'refrigerator', 'book', 'clock', 'vase', 'scissors',
               'teddy bear', 'hair drier', 'toothbrush']


async def load_image(request) -> ByteString:
    """
    Get a image from multipart upload
    :param request: `aiohttp.web.BaseRequest`
    :return: a byte string got from request
    """
    reader = await request.multipart()
    part = await reader.next()
    content_type = part.headers[hdrs.CONTENT_TYPE]
    if 'image' in content_type:
        image_binary = await part.read(decode=True)
        img = Image.open(io.BytesIO(image_binary))
        np_array = np.array(img)
        return np_array
    else:
        raise web.HTTPBadRequest(
            reason=f'incorrect payload format {content_type}')

def gerp():
    return "yert"


async def classify_handler(request):
    #image = skimage.io.imread(os.path.join(IMAGE_DIR, 'Charnemea.jpeg'))
    # Run detection
    image = await load_image(request)
    #loop = asyncio.get_event_loop()
    mask = model.detect([image], verbose=1)
    #mask = loop.run_in_executor(executor, gerp())
    #loop.close()
    # Visualize results
    r = mask[0]
    result = await visualize.save_instances(image, r['rois'], r['masks'], r['class_ids'], 
                                class_names, r['scores'])
    #result = await visualize.mask_on(image, r['rois'], r['masks'], r['class_ids'], 
    #                            class_names, r['scores'])
    data = result
    #return web.Response(status=200, content_type='application/json')
    return web.FileResponse(ROOT_DIR + '/Mask_RCNN/results/temporary.png', headers={
        hdrs.CONTENT_TYPE: "image/png"})

def build_app():
    app = web.Application(
    )
    app.router.add_route('POST', '/',
                         classify_handler, name='classifier')
    #app.add_routes([web.get('/', receiver)])
    return app


app = build_app()
logger = logging.getLogger(__name__)

executor = ThreadPoolExecutor(5)
if __name__ == '__main__':

    # Create a logger object.
    logging.basicConfig(
        level=LOGGING_LEVEL,
        format=('[%(asctime)s] {%(filename)s:%(lineno)d} '
                '%(levelname)s - %(message)s'),
    )
    web.run_app(app, host='0.0.0.0', port=6660)


Using TensorFlow backend.
